# CNN Model 1

## The approach:

### Without time feature
### Pass 1x29 vectors into a convolutional layer, with kernel size 29, with some D number of filters
### Add extra conv and dense layer to the model to see the effect

## CNN1: single conv layer single dense layer:

#### Imports, data set up 

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import numpy
from numpy.random import seed
import pandas as pd
import numpy as np
from sklearn.preprocessing import minmax_scale
from keras.layers.convolutional import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import keras

data = pd.read_csv("creditcard.csv")


# Normalise and reshape the Amount column, so it's values lie between -1 and 1
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# data['norm_Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1,1))

# Assign variables x and y corresponding to row data and it's class value
X = data.loc[:, data.columns != 'Class']
y = data.loc[:, data.columns == 'Class']

data_X_cols = X.columns
data_y_cols = y.columns

# Scale data using StandardScaler
pipeline = Pipeline([
('scaling', StandardScaler()),
])
preprocessor = pipeline.fit(X)
data = preprocessor.transform(X)
data = pd.DataFrame(data, columns=data_X_cols)
data['Class'] = y

# RE - Assign variables x and y corresponding to row data and it's class value
X = data.loc[:, data.columns != 'Class']
y = data.loc[:, data.columns == 'Class']



Using TensorFlow backend.


In [33]:
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.996583,-0.694242,-0.044075,1.672773,0.973366,-0.245117,0.347068,0.193679,0.082637,0.331128,...,-0.024923,0.382854,-0.176911,0.110507,0.246585,-0.392170,0.330892,-0.063781,0.244964,0
1,-1.996583,0.608496,0.161176,0.109797,0.316523,0.043483,-0.061820,-0.063700,0.071253,-0.232494,...,-0.307377,-0.880077,0.162201,-0.561131,0.320694,0.261069,-0.022256,0.044608,-0.342475,0
2,-1.996562,-0.693500,-0.811578,1.169468,0.268231,-0.364572,1.351454,0.639776,0.207373,-1.378675,...,0.337632,1.063358,1.456320,-1.138092,-0.628537,-0.288447,-0.137137,-0.181021,1.160686,0
3,-1.996562,-0.493325,-0.112169,1.182516,-0.609727,-0.007469,0.936150,0.192071,0.316018,-1.262503,...,-0.147443,0.007267,-0.304777,-1.941027,1.241904,-0.460217,0.155396,0.186189,0.140534,0
4,-1.996541,-0.591330,0.531541,1.021412,0.284655,-0.295015,0.071999,0.479302,-0.226510,0.744326,...,-0.012839,1.100011,-0.220123,0.233250,-0.395202,1.041611,0.543620,0.651816,-0.073403,0
5,-1.996541,-0.217475,0.581675,0.752585,-0.118833,0.305009,-0.022313,0.384936,0.217955,-0.517619,...,-0.283522,-0.771427,-0.042273,-0.613273,-0.446584,0.219637,0.628900,0.245636,-0.338556,0
6,-1.996499,0.627795,0.085389,0.029923,0.849383,0.139020,0.204695,-0.004170,0.067998,0.423218,...,-0.228334,-0.373032,-0.246780,-1.287973,1.439037,-0.533436,0.085492,0.015656,-0.333279,0
7,-1.996436,-0.328928,0.858692,0.708576,-0.347631,0.687512,0.321345,0.905860,-3.188229,0.560129,...,2.645889,-1.399276,0.092085,-1.072754,-0.796633,-0.107075,-2.990154,-3.288083,-0.190107,0
8,-1.996436,-0.456573,0.173291,-0.074653,-0.191774,1.934149,2.793594,0.299206,0.712592,-0.356851,...,-0.099963,-0.369425,-0.327055,1.670269,0.715943,-0.796633,0.029104,0.431420,0.019392,0
9,-1.996394,-0.172698,0.678005,0.688781,-0.156927,0.361792,-0.185219,0.526706,0.058223,-0.670587,...,-0.336156,-0.873298,-0.193438,-0.635767,-0.133773,0.195342,0.610010,0.251681,-0.338516,0


#### Function to create and return CNN1 model

In [34]:
# Function to create model
def create_model():
    # create model
    seed(2017)
    conv = Sequential()
    conv.add(Conv1D(256, 30, input_shape=(30, 1), activation='relu'))
    conv.add(Flatten())
    conv.add(Dense(300, activation = 'relu'))
    conv.add(Dense(2, activation = 'softmax'))

    sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
    
    # Compile model
    conv.compile(loss='categorical_crossentropy', optimizer=sgd)
    return conv
 

In [35]:
# Setting up dataframe table properties
log_cols=["Name", "F1 Score", "Precision", "Recall", "Training Time"]
log = pd.DataFrame(columns=log_cols)

#### Function to smote data, from before

In [36]:
def smote_data(x_data, y_data):
    from collections import Counter
    from imblearn.over_sampling import SMOTE
    sm = SMOTE()
    X_res, y_res = sm.fit_sample(x_data, y_data)
    print('Resampling the data with SMOTE. . .')
    print('Resampled training dataset shape {}'.format(Counter(y_res)))

    return X_res, y_res

#### This custom cross val function follows the same method as before but with a functional parameter create_model to allow passing various CNN model creating functions and hence reuse this code

In [37]:
def custom_cross_val(X, y, create_model, n):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.base import clone
    import datetime
    from sklearn.metrics import precision_recall_fscore_support
    
    print 'Cross validating... \n'
    skfolds = StratifiedKFold(n_splits=n, random_state=42)
    
    precision = []
    recall = []
    f1score = []
    elapsed_times = []
    cv = 0
    model_to_save = []
    
    for train_index, test_index in skfolds.split(X, y):
        cv=cv+1
        print len(train_index)
        clone_clf = create_model()
        X_train_folds = X.iloc[train_index]
        y_train_folds = y.iloc[train_index]
        X_test_fold = X.iloc[test_index]
        y_test_fold = y.iloc[test_index]
        
        print len(y_train_folds[y_train_folds['Class']==1])
        X_res, y_res = smote_data(X_train_folds, y_train_folds )
        
        print X_res.shape, type(X_res)
        print y_res.shape

        X_train = X_res.reshape(X_res.shape[0], 30, 1)
        Y_train = y_res.reshape(y_res.shape[0], 1)
        X_test = X_test_fold.values.reshape(X_test_fold.values.shape[0], 30, 1)
        Y_test = y_test_fold.values.reshape(y_test_fold.values.shape[0], 1)

        Y_test = keras.utils.to_categorical(Y_test)
        Y_train = keras.utils.to_categorical(Y_train)
        print Y_test.shape
        print Y_train.shape
        
        
        start = datetime.datetime.now()
        
        print('Fitting the model... CV[{}]'.format(cv)) 
        
        clone_clf.fit(X_train, Y_train, batch_size = 500, epochs = 25, verbose =1)
        model_to_save = clone_clf
        
        end = datetime.datetime.now()
        elapsed = end - start
        elapsed_times.append(elapsed)
        
        y_pred = clone_clf.predict(X_test)
        
        # Set cut off point for class boundaries
        cutt_off_tr = 0.5
        y_pred[np.where(y_pred>=cutt_off_tr)] = 1
        y_pred[np.where(y_pred<cutt_off_tr)]  = 0
        
        prfs = precision_recall_fscore_support(Y_test, y_pred, labels=[0])
        
        precision.append(prfs[0][1])
        recall.append(prfs[1][1])
        f1score.append(prfs[2][1])  
    
    model_to_save.save('CNNv1.h5')
    
    average_timedelta = sum(elapsed_times, datetime.timedelta(0)) / len(elapsed_times)
    entry = ['CNN Model 1', np.mean(f1score), np.mean(precision), np.mean(recall), average_timedelta]
    print('Mean scores: ', entry )
    return entry

In [38]:
results = custom_cross_val(X, y, create_model, 3)

log_entry = pd.DataFrame([results], columns=log_cols)
log = log.append(log_entry)

# Replace table index by the Classifier column
log.set_index('Name', inplace=True)


Cross validating... 

189871
328
Resampling the data with SMOTE. . .
Resampled training dataset shape Counter({0: 189543, 1: 189543})
(379086, 30) <type 'numpy.ndarray'>
(379086,)
(94936, 2)
(379086, 2)
Fitting the model... CV[1]
Epoch 1/25
379086/379086 [==============================] - 5s 14us/step - loss: 0.0272
Epoch 2/25
379086/379086 [==============================] - 5s 13us/step - loss: 0.0049
Epoch 3/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0038
Epoch 4/25
379086/379086 [==============================] - 4s 12us/step - loss: 0.0030
Epoch 5/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0026
Epoch 6/25
379086/379086 [==============================] - 4s 12us/step - loss: 0.0021
Epoch 7/25
379086/379086 [==============================] - 4s 12us/step - loss: 0.0021
Epoch 8/25
379086/379086 [==============================] - 5s 12us/step - loss: 0.0021
Epoch 9/25
379086/379086 [==============================] - 4s 11u

## CNN1.2: Added Conv and Dense layer

In [2]:
# Function to create model, required for KerasClassifier
def create_model_2():
    # create model
    seed(2017)
    conv = Sequential()
    conv.add(Conv1D(256, 30, input_shape=(30, 1), activation='relu'))
    conv.add(Conv1D(256, 1, activation='relu'))
    conv.add(Flatten())

    conv.add(Dense(300, activation = 'relu'))
    conv.add(Dense(100, activation = 'relu'))
    conv.add(Dense(2, activation = 'softmax'))

    sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
    
    # Compile model
    conv.compile(loss='categorical_crossentropy', optimizer=sgd)
    return conv
 

In [40]:
results1_2 = custom_cross_val(X, y, create_model_2, 3)
results1_2[0] = 'CNN Model 1.2'


Cross validating... 

189871
328
Resampling the data with SMOTE. . .
Resampled training dataset shape Counter({0: 189543, 1: 189543})
(379086, 30) <type 'numpy.ndarray'>
(379086,)
(94936, 2)
(379086, 2)
Fitting the model... CV[1]
Epoch 1/25
379086/379086 [==============================] - 10s 27us/step - loss: 0.0238
Epoch 2/25
379086/379086 [==============================] - 11s 28us/step - loss: 0.0066
Epoch 3/25
379086/379086 [==============================] - 13s 34us/step - loss: 0.0032
Epoch 4/25
379086/379086 [==============================] - 11s 30us/step - loss: 0.0025
Epoch 5/25
379086/379086 [==============================] - 10s 27us/step - loss: 0.0023
Epoch 6/25
379086/379086 [==============================] - 10s 26us/step - loss: 0.0023
Epoch 7/25
379086/379086 [==============================] - 9s 25us/step - loss: 0.0021
Epoch 8/25
379086/379086 [==============================] - 9s 24us/step - loss: 0.0019
Epoch 9/25
379086/379086 [==============================] - 

In [41]:
log_entry = pd.DataFrame([results], columns=log_cols)
log = log.append(log_entry)
log_entry = pd.DataFrame([results1_2], columns=log_cols)
log = log.append(log_entry)

In [42]:
log

,F1 Score,Name,Precision,Recall,Training Time
CNN Model 1,0.603035,NaN,0.532014,0.825203,00:01:51.786324
0,0.603035,CNN Model 1,0.532014,0.825203,00:01:51.786324
0,0.647514,CNN Model 1.2,0.577108,0.821138,00:03:58.754256


In [43]:
# Replace table index by the Classifier column
log.set_index('Name', inplace=True)

In [44]:
log

,F1 Score,Precision,Recall,Training Time
Name,,,,
NaN,0.603035,0.532014,0.825203,00:01:51.786324
CNN Model 1,0.603035,0.532014,0.825203,00:01:51.786324
CNN Model 1.2,0.647514,0.577108,0.821138,00:03:58.754256


## Results

In [45]:
print log

               F1 Score  Precision    Recall   Training Time
Name                                                        
NaN            0.603035   0.532014  0.825203 00:01:51.786324
CNN Model 1    0.603035   0.532014  0.825203 00:01:51.786324
CNN Model 1.2  0.647514   0.577108  0.821138 00:03:58.754256


In [46]:
import pickle
pickle.dump( log , open( "cnnv1_results.p", "wb" ) )

## Model Summaries

In [47]:
conv1 = create_model()
conv1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_27 (Conv1D)           (None, 1, 256)            7936      
_________________________________________________________________
flatten_20 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 300)               77100     
_________________________________________________________________
dense_47 (Dense)             (None, 2)                 602       
Total params: 85,638
Trainable params: 85,638
Non-trainable params: 0
_________________________________________________________________


In [48]:
conv2 = create_model_2()
conv2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_28 (Conv1D)           (None, 1, 256)            7936      
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 1, 256)            65792     
_________________________________________________________________
flatten_21 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 300)               77100     
_________________________________________________________________
dense_49 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_50 (Dense)             (None, 2)                 202       
Total params: 181,130
Trainable params: 181,130
Non-trainable params: 0
_________________________________________________________________


## Evaluation

We see that in this simple CNN approach, we do not meet the best we have seen amongst baseline models. However, looking a bit more specifically, we see that Recall scores are comparable, but without the drastic low in precision scores, like we had in some baseline models. 

Comparing specifically with the (tuned) Random Forest classifier:
We see that the Recall is approximately the same, it is the precision that is lower.

Comparing with the Smote results across all baseline models we can see that our CNN models are higher in F1-score except the RF classifier. In particular the precision scores are comparable to the second best performing classifier the MLP. 

In going from CNN1 to CNN1.2, we see that the major performance increase comes from raising the precision score, resulting in a higher F1 overall. This means that our approach of adding an extra conv layer and dense layer helped to improve precision of the model. The training time however, obviously took longer, almost double the time.

## Same as before, but trying higher class threshold

In [49]:
def custom_cross_val_2(X, y, create_model, n):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.base import clone
    import datetime
    from sklearn.metrics import precision_recall_fscore_support
    
    print 'Cross validating... \n'
    skfolds = StratifiedKFold(n_splits=n, random_state=42)
    
    precision = []
    recall = []
    f1score = []
    elapsed_times = []
    cv = 0
    
    for train_index, test_index in skfolds.split(X, y):
        cv=cv+1
        print len(train_index)
        clone_clf = create_model()
        X_train_folds = X.iloc[train_index]
        y_train_folds = y.iloc[train_index]
        X_test_fold = X.iloc[test_index]
        y_test_fold = y.iloc[test_index]
        
        print len(y_train_folds[y_train_folds['Class']==1])
        X_res, y_res = smote_data(X_train_folds, y_train_folds )
        
        print X_res.shape, type(X_res)
        print y_res.shape

        X_train = X_res.reshape(X_res.shape[0], 30, 1)
        Y_train = y_res.reshape(y_res.shape[0], 1)
        X_test = X_test_fold.values.reshape(X_test_fold.values.shape[0], 30, 1)
        Y_test = y_test_fold.values.reshape(y_test_fold.values.shape[0], 1)

        Y_test = keras.utils.to_categorical(Y_test)
        Y_train = keras.utils.to_categorical(Y_train)
        print Y_test.shape
        print Y_train.shape
        
        
        start = datetime.datetime.now()
        
        print('Fitting the model... CV[{}]'.format(cv)) 
        
        clone_clf.fit(X_train, Y_train, batch_size = 500, epochs = 25, verbose =1)
        end = datetime.datetime.now()
        elapsed = end - start
        elapsed_times.append(elapsed)
        
        y_pred = clone_clf.predict(X_test)
        
        # Set cut off point for class boundaries
        cutt_off_tr = 0.6
        y_pred[np.where(y_pred>=cutt_off_tr)] = 1
        y_pred[np.where(y_pred<cutt_off_tr)]  = 0
        
        prfs = precision_recall_fscore_support(Y_test, y_pred, labels=[0])
        
        precision.append(prfs[0][1])
        recall.append(prfs[1][1])
        f1score.append(prfs[2][1])  
    
    average_timedelta = sum(elapsed_times, datetime.timedelta(0)) / len(elapsed_times)
    entry = ['CNN Model 1.1', np.mean(f1score), np.mean(precision), np.mean(recall), average_timedelta]
    print('Mean scores: ', entry )
    return entry

In [50]:
results2 = custom_cross_val(X, y, create_model, 3)

Cross validating... 

189871
328
Resampling the data with SMOTE. . .
Resampled training dataset shape Counter({0: 189543, 1: 189543})
(379086, 30) <type 'numpy.ndarray'>
(379086,)
(94936, 2)
(379086, 2)
Fitting the model... CV[1]
Epoch 1/25
379086/379086 [==============================] - 5s 14us/step - loss: 0.0274
Epoch 2/25
379086/379086 [==============================] - 4s 12us/step - loss: 0.0046
Epoch 3/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0045
Epoch 4/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0028
Epoch 5/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0024
Epoch 6/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0020
Epoch 7/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0019
Epoch 8/25
379086/379086 [==============================] - 4s 11us/step - loss: 0.0019
Epoch 9/25
379086/379086 [==============================] - 4s 11u

In [3]:
model = create_model_2()

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1, 256)            7936      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 256)            65792     
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               77100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 181,130
Trainable params: 181,130
Non-trainable params: 0
_________________________________________________________________
